# Jobathon Nov 2022 Time series Model

## Table of Contents
### 1. [Read Train and Test ](#read)
### 2. [Feature Generation](#feature)
### 3. [Train and Validation Split](#split)
### 4. [Model Evaluation using Facebook Prophet](#model_eval_fbprophet)
### 4. [Model Evaluation using Thyme Boost](#model_eval_thyme)
### 4. [Model Evaluation using Unobserved Components](#model_eval)
### 5. [Model Finalization for Test Prediction](#model_final)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
pd.options.display.max_columns=500
pd.options.display.max_rows=500

In [ ]:
from pandas.tseries.holiday import *

In [ ]:
# !pip install pmdarima

In [ ]:
# !pip install ThymeBoost

In [ ]:
KAGGLE=True

In [ ]:
if KAGGLE:
    path = '/kaggle/input/jobathon-nov-2022/'
else:
    path = 'input/'

<a id='read'></a>
## Read Train and Test Data

In [ ]:
train=pd.read_csv(path+'train.csv')
print(train.shape)

In [ ]:
test=pd.read_csv(path+'test.csv')
print(test.shape)

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train['energy'].describe()
targetcol='energy'

In [ ]:
train['datetime']=pd.to_datetime(train['datetime'],infer_datetime_format=True)
test['datetime']=pd.to_datetime(test['datetime'],infer_datetime_format=True)

In [ ]:
train['datetime'].min(),train['datetime'].max()

<a id='feature'></a>
## Feature Generation

In [ ]:
#create hour map based on business, non - business , sleeping hours etc.
hour_map={0:0,1:0,2:0,3:0,4:0,5:0,
         6:1,7:1,8:1,
         9:2,10:2,11:2,
         12:3,13:3,14:3,15:3,
         16:4,17:4,
         18:5,19:5,20:5,
         21:6,22:6,23:6}
 
#Monday and Sunday as group 1, Saturday as group 2, Otherdays as group 3
dayofweek_map = {0:1,6:1,
                5:2,
                1:3,2:3,3:3,4:3}

create basic date related features

In [ ]:
def gen_datefeats(data):
    data['year']=data['datetime'].dt.year
    data['month']=data['datetime'].dt.month
    data['day']=data['datetime'].dt.day
    data['hour']=data['datetime'].dt.hour
    data['weekofyear']=data['datetime'].dt.isocalendar().week
    data['dayofweek']=data['datetime'].dt.dayofweek
    data['dayofweek_grp']=data['dayofweek'].replace(dayofweek_map)
    data['quarter']=data['datetime'].dt.quarter
    data['is_weekend']=data['datetime'].dt.dayofweek > 4
    data['day_part']=data['hour'].replace(hour_map)

In [ ]:
gen_datefeats(train)
gen_datefeats(test)

create holiday features with special holiday denoting christmas long holidays

In [ ]:
#generate holidays feature
def gen_holiday_feat(data,start,end):
    cal = USFederalHolidayCalendar()
    holiday_dates = cal.holidays(start=start, end=end)
    data['is_holiday'] = False
    mask = data['datetime'].dt.date.astype('datetime64').isin(holiday_dates)
    data.loc[mask,'is_holiday']=True   
    
    data['special_holiday']=False
    mask= ((data['datetime'].dt.month==12) & (data['datetime'].dt.day>=24))  \
           | ((data['datetime'].dt.month==1) & (data['datetime'].dt.day<3)) 
    data.loc[mask,'special_holiday']=True   
        
    return holiday_dates

In [ ]:
holidays= gen_holiday_feat(train,train['datetime'].dt.date.min(),train['datetime'].dt.date.max())
print(holidays)
print(train['special_holiday'].value_counts())
train['is_holiday'].value_counts()

In [ ]:
train.head()

generate hour aggregate features month-wise, quarter-wise, week of year-wise and week day group wise

In [ ]:
def gen_datetime_comb_feats(data):
    data['dayofweek_hr']=data['dayofweek'].astype('str') + '_'+data['hour'].astype('str')
    data['weekofyear_hr']=data['weekofyear'].astype('str') + '_'+data['hour'].astype('str')
    data['month_hr']=data['month'].astype('str') + '_'+data['hour'].astype('str')
    data['quarter_hr']=data['quarter'].astype('str') + '_'+data['hour'].astype('str')


def gen_mean_feats(train,test,cols,newcolname):
    grouped=train.groupby(cols)[targetcol].mean().reset_index()
    grouped.columns=cols+[newcolname]
    train=train.merge(grouped,on=cols)    
    test=test.merge(grouped,on=cols)   
    return train,test
    
def gen_mean_feats_all(train,test):
    train,test=gen_mean_feats(train,test,['month','hour'],'month_hour_mean')
    train,test=gen_mean_feats(train,test,['quarter','hour'],'quarter_hour_mean')
    train,test=gen_mean_feats(train,test,['weekofyear','hour'],'weekofyear_mean')
    train,test=gen_mean_feats(train,test,['dayofweek_grp','hour'],'dayofweek_grp_mean')  
    
    train.sort_values('datetime',inplace=True)
    train.reset_index(drop=True,inplace=True)
    test.sort_values('datetime',inplace=True)
    test.reset_index(drop=True,inplace=True)
    return train,test

In [ ]:
train,test=gen_mean_feats_all(train,test)

In [ ]:
gen_datetime_comb_feats(train)
gen_datetime_comb_feats(test)

In [ ]:
train.head(10)

In [ ]:
test.head()

In [ ]:
train['year'].unique()

In [ ]:
test['datetime'].min(),test['datetime'].max()

In [ ]:
holidays= gen_holiday_feat(test,test['datetime'].dt.date.min(),test['datetime'].dt.date.max())
print(len(holidays))
print(holidays)
print(test['special_holiday'].value_counts())
test['is_holiday'].value_counts()

In [ ]:
targetcol = 'energy'

In [ ]:
# train[targetcol].fillna(train[targetcol].mean(),inplace=True)
train[targetcol].fillna(method='ffill',inplace=True)

Create Lag Features

In [ ]:
def create_lag(data,lagno_list):
    res = pd.DataFrame()
    for i in lagno_list:
        shifted = data.shift(i)
        res=pd.concat([res,shifted],axis=1)

#     res=pd.concat([data.shift(i) for i in lagno_list],axis=1)
    res.columns=[f'lag_{i}' for i in lagno_list]
    return res

In [ ]:
test.shape

In [ ]:
train['istrain']=1
test['istrain']=0
combined = pd.concat([train,test],axis=0) 
#lag 1 year, 3 year, quarter, month,week
lag_df = create_lag(combined[targetcol],[24,168,720,2160,8760,26304])
combined=pd.concat([combined,lag_df],axis=1)
train=combined[combined['istrain']==1]
test=combined[combined['istrain']==0]

del combined,train['istrain'],test['istrain'],test[targetcol]
print(train.shape,test.shape)

In [ ]:
train.head()

In [ ]:
train.columns

Fill null values in train data using previous hour values

In [ ]:
cols = [col for col in train.columns if col.startswith('lag_')]
target_mean = train[targetcol].mean()
for col in cols:
    train[col].fillna(0,inplace=True)

<a id='split'></a>
## Train and Validation Split

Validation Set from 2016 to 2018 <br>
Train Set from 2008 to 2015

In [ ]:
import datetime 
train_start = datetime.datetime(year=2008,month=1,day=1,hour=0)
val_start = datetime.datetime(year=2016,month=1,day=1,hour=0)
val_end = datetime.datetime(year=2018,month=12,day=31,hour=23)

X_val= train[(train['datetime']>=val_start) & (train['datetime']<=val_end)].copy()
X_train= train[(train['datetime']>=train_start) & (train['datetime']<val_start)].copy()
print(X_train.shape)
print(X_val.shape)
X_val.head()           

In [ ]:
val_target = X_val[targetcol]
print(X_val[targetcol].isnull().sum())

Create simple validation prediction baseline using train mean value 

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
#compute baseline error by predicting train energy mean as the energy for all time
val_preds_baseline = np.full(len(X_val),train[targetcol].mean())

In [ ]:
#compute error score on baseline predictions
val_score = mean_squared_error(val_target,val_preds_baseline,squared=False)
print('valid score:',val_score)

<a id='model_eval_fbprophet'></a>
## Model Evaluation using Facebook Prophet

In [ ]:
!pip install pystan==2.19.1.1

In [ ]:
!pip install fbprophet

In [ ]:
from fbprophet import Prophet
from fbprophet.plot import plot_plotly
from fbprophet.plot import plot_plotly
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
from fbprophet.plot import plot_cross_validation_metric

In [ ]:
from multiprocessing import cpu_count
from joblib import Parallel, delayed
from multiprocessing import cpu_count

In [ ]:
from sklearn.model_selection import ParameterGrid

In [ ]:
def tune_prophet_params(param):
    val_start   = 68688
    val_period  = 26304
    print(param)
    np.random.seed(0)
    train_model =Prophet(uncertainty_samples=0,
                        changepoint_prior_scale = param['changepoint_prior_scale'],
#                              n_changepoints = param['n_changepoints'],
                         changepoint_range = param['changepoint_range'],
                         weekly_seasonality=True,
                         daily_seasonality = True,
                         yearly_seasonality = True,
                         interval_width=0.95)
    train_model.fit(train_df[:val_start])
    future_df = train_model.make_future_dataframe(periods=val_period, freq='H',include_history = False)
#     future_df['cap']=4000
#     future_df['floor']=1000
    train_forecast = train_model.predict(future_df)
    val=train_forecast[['ds','yhat']]
    Actual = train_df[val_start:val_start+val_period]
    RMSE = mean_squared_error(Actual['y'],abs(val['yhat']),squared=False)
    print('RMSE------------------------------------',RMSE)
    model_param = {'RMSE':RMSE}
    model_param.update(param)

    return model_param

Evaluation of Best Tuned Model with additional regressors

In [ ]:
def add_regressors(data_prophet,data_orig,regressors):
    df_with_reg = pd.concat([data_prophet.reset_index(drop=True),
                             data_orig[regressors].head(len(data_prophet)).reset_index(drop=True)],axis=1)
    return df_with_reg

In [ ]:
# cols = ['hour','dayofweek','weekofyear','quarter_hr','dayofweek_hr','month_hr']
cols = ['hour','dayofweek','quarter','quarter_hr','month_hr']
exog_train = pd.get_dummies(X_train.set_index('datetime')[cols],columns=cols,prefix=cols)
exog_test = pd.get_dummies(X_val.set_index('datetime')[cols],columns=cols,prefix=cols)

y_train = X_train.set_index('datetime')[targetcol].copy()
y_test = X_val.set_index('datetime')[targetcol].copy()

In [ ]:
# regressors = list(exog_train.columns)
regressors = [col for col in exog_train.columns if col.startswith('hour_')]
X_train_df = pd.concat([exog_train,y_train],axis=1)
X_train_df = X_train_df.reset_index().rename(columns={'datetime': 'ds', 
                        targetcol: 'y'})
print(X_train_df.shape)
X_train_df.head()

In [ ]:
X_val_df = pd.concat([exog_test,y_test],axis=1)
X_val_df = X_val_df.reset_index().rename(columns={'datetime': 'ds', 
                        targetcol: 'y'})
print(X_val_df.shape)
X_val_df.head()

In [ ]:
%%time
val_start   = len(X_train_df)#10000
val_period  = len(X_val_df)#48
# val_start   = 68688
# val_period  = 26304
np.random.seed(100)
train_model =Prophet(uncertainty_samples=0,
                     mcmc_samples = 100,
                    changepoint_prior_scale =0.1,
#                     n_changepoints = 50,
                     changepoint_range = 0.9,
                     weekly_seasonality=True,
                     daily_seasonality = True,
                     yearly_seasonality = True,
                     interval_width=0.95)

# train_model.add_seasonality(name='daily', period=24, fourier_order=15, prior_scale=0.1)
# train_model.add_seasonality(name='weekly', period=168, fourier_order=3, prior_scale=0.1)
# train_model.add_seasonality(name='yearly', period=8760, fourier_order=3, prior_scale=0.01)

# RMSE: 683 SCALE: 50,15,2 ORDEr: 15,3,3
# RMSE: 659 SCALE: 15,10,2 ORDEr: 15,3,3
# RMSE: 445 SCALE: 1,1,0.1 ORDER: 15,3,3
# RMSE: 440 SCALE: 0.1,0.1,0.01 ORDER: 15,3,3

# RMSE: 250 mcmc_samples=50, adapt_delta: 0.85
# RMSE: 245 mcmc_samples=50, adapt_delta: 0.99
# RMSE: 227 mcmc_samples=50, adapt_delta: 0.99 , chains=1 (time: 4 min)
# RMSE: 225 mcmc_samples=50, adapt_delta: 0.99 , chains=1 with hour regressors (time: 4 min)
# RMSE: 225 changepoint_prior_scale=0.1, mcmc_samples=50, adapt_delta: 0.99 , chains=1 (time: 1 min)
# RMSE: 222 changepoint_prior_scale=0.5, mcmc_samples=50, adapt_delta: 0.99 , chains=1 (time: 1 min)


# RMSE: 284 mcmc_samples=100, adapt_delta: 0.99 , chains=2 (time: 30 min)

# train_w_reg = add_regressors(train_df[:val_start],X_train,regressors)

In [ ]:
# for col in regressors:
#     train_model.add_regressor(col,prior_scale=2,standardize=False)

In [ ]:
%%time
train_model.fit(X_train_df[:val_start],
                seed=100,chains=4,
                control={'adapt_delta': 0.99,
#                          'max_treedepth': 20
                        })

In [ ]:
# future_df = train_model.make_future_dataframe(periods=val_period, freq='H',include_history = False)
# X_val_df['cap']=4000
# X_val_df['floor']=1000
# future_w_reg = add_regressors(future_df,X_val,regressors)
np.random.seed(100)
train_forecast = train_model.predict(X_val_df[:val_period])
RMSE = mean_squared_error(X_val_df[:val_period]['y'],abs(train_forecast['yhat']),squared=False)
# RMSE = mean_squared_error(X_val_df['y'],abs(train_forecast['yhat']),squared=False)
print('RMSE------------------------------------',RMSE)

In [ ]:
from fbprophet.serialize import model_to_json, model_from_json

with open('fbprophet_mcmc100_w_reg.json', 'w') as fout:
    fout.write(model_to_json(train_model))  # Save model

with open('fbprophet_mcmc100_w_reg.json', 'r') as fin:
    temp_model = model_from_json(fin.read())  # Load model

In [ ]:
train_forecast[['yhat']].to_csv('preds.csv',index=False)

In [ ]:
train_forecast['yhat']